In [1]:
import pandas as pd
import re

df = pd.read_csv("data (3).csv", index_col="Unnamed: 0", encoding="UTF-8")

df.head()
new_df = pd.DataFrame(columns=['genre', 'lyrics', 'first_line'])

for i, row in df.iterrows():
    lyrics = re.sub(r'[^a-zA-Zа-яА-ЯёЁ\s.,!?[]]', '', row['lyrics'])
    lyrics = re.sub(r"You might also like", "", lyrics)
    if isinstance(lyrics, str):
        for chord in lyrics.split('\n\n'):
            text = chord.split('\n')
            text = [i for i in text if i != '']
            for i in range(len(text)//4):
                new_text = text[i*4]+'\n'+text[i*4+1]+'\n'+text[i*4+2]+'\n'+text[i*4+3]+'\n'
                new_df = pd.concat([new_df, pd.DataFrame([[row["genre"], new_text, text[i*4]]], columns=['genre', 'lyrics', 'first_line'])])

new_df = new_df.drop_duplicates()

def convert_to_json(data):
    return {
"request": [
    {
      "role": "system",
      "text": "Вы известный поэт с высочайшим навыком рифмования, а так же известный музыкальный исполнитель.\nВаша задача придумать осмысленное четверостишье, в котором будут читаться рифмы в определенном жанре.\nВам будет дана первая строка четверостишья, а так же его жанр. Вы должны вернуть только текст четверостишья."
    },
    {
      "role": "user",
      "text": f"Первая строка: {data['first_line']}\nЖанр: [{data['genre']}]"
    }
  ],
  "response": data["lyrics"]
}


new_df["json"] = new_df.apply(convert_to_json, axis=1)

import json

with open('output.json', 'w') as outfile:
    for entry in list(new_df['json'].values):
        json.dump(entry, outfile, ensure_ascii=False)
        outfile.write('\n')
    
    




/Users/aleksandrlazarev/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
